# Задание 1

1) Это задача Name entity recognition()

# Задание 2
Реализуйте чтение датасета в pandas DataFrame с обязательными колонками "document_id", "document_text", "entity", "gold_answer". 

In [16]:
from corus import load_bsnlp
path = '../data'
doc_id = []
doc_text = []
entity = []
gold_answer = []
records = load_bsnlp(path)
for record in records:
    for ner in record.substrings:
        doc_id.append(record.id)
        doc_text.append(record.text)
        entity.append(ner.type)
        gold_answer.append(ner.normal)
# record = next(records)
# record

In [17]:
record

BsnlpMarkup(
    id='ru-1017',
    name='brexit_ru.txt_file_1017',
    lang='ru',
    date=datetime.datetime(2018, 7, 9, 0, 0),
    url='http://www.unn.com.ua/ru/news/1740316-boris-dzhonson-podav-u-vidstavku-z-posta-ministra-zakordonnikh-sprav-velikoyi-britaniyi',
    text='Борис Джонсон подал в отставку с поста министра иностранных дел Великобритании\n\nКИЕВ. 9 июля. УНН. Борис Джонсон подал в отставку с поста министра иностранных дел Великобритании. Об этом сообщает The Guardian, передает УНН. "Сегодня днем премьер-министр принял отставку Бориса Джонсона с поста качестве министра иностранных дел. Его замена будет объявлена в ближайшее время. Премьер-министр благодарит Бориса за его работу", - сообщил пресс-секретарь британского парламента. Таким образом, Джонсон стал третьим министром, который покинул свой пост в течении суток. Ранее о своем уходе объявили министр по вопросам выхода Великобритании из ЕС Дэвид Дэвис и заместитель министра по Brexit Стив Бейкер. Напомним, ранее премьер

In [18]:
import pandas as pd

data = pd.DataFrame({
    'document_id': doc_id,
    'document_text': doc_text,
    'entity': entity,
    'gold_answer': gold_answer
})

In [19]:
data

,document_id,document_text,entity,gold_answer
0,ru-10,Тереза Мэй рассчитывает усидеть в седле до зав...,EVT,Brexit
1,ru-10,Тереза Мэй рассчитывает усидеть в седле до зав...,LOC,Альбион
2,ru-10,Тереза Мэй рассчитывает усидеть в седле до зав...,LOC,Альбион
3,ru-10,Тереза Мэй рассчитывает усидеть в седле до зав...,PER,Борис Джонсон
4,ru-10,Тереза Мэй рассчитывает усидеть в седле до зав...,LOC,Британия
...,...,...,...,...
148,ru-1017,Борис Джонсон подал в отставку с поста министр...,LOC,КИЕВ
149,ru-1017,Борис Джонсон подал в отставку с поста министр...,ORG,МИД Соединенного Королевства
150,ru-1017,Борис Джонсон подал в отставку с поста министр...,PER,Стив Бейкер
151,ru-1017,Борис Джонсон подал в отставку с поста министр...,PER,Тереза Мэй
